In [11]:
import nltk
import pandas as pd
import numpy as np
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import spacy
import re, string

In [4]:
!ls

airbnb_listings_descriptions.xlsx  airbnb_text_reviews.xlsx  drive  sample_data


In [5]:
reviews = pd.read_excel('/content/airbnb_text_reviews.xlsx')

In [6]:
reviews.sample(1)

,listing_id,id,date,reviewer_id,reviewer_name,comments
372600,12628500,177695520,2017-08-03,47826904,Chaiwat,Excellent host! Dante is very accommodating. T...


In [7]:
reviews.sample(3)

,listing_id,id,date,reviewer_id,reviewer_name,comments
60219,546383,53145072,2015-11-05,45856935,Cristian,A nice place close to a very dynamic area and ...
267280,7276534,173635576,2017-07-23,141449582,Kaho,Very gracious host and a safe neighborhood! Hi...
741012,33267781,439198118,2019-04-18,1279817,Jeffrey,A great apartment in one of the best locations...


In [8]:
reviews['comments'] = reviews['comments'].astype('string')

In [9]:
reviews['comments'].describe()

count         1048358
unique        1000410
top       Great place
freq             1366
Name: comments, dtype: object

## removing rows with only punctuation

In [12]:
pattern = re.compile("[\d{}]+$".format(re.escape(string.punctuation)))

In [13]:
reviews.loc[(reviews['comments'] == '.') | (reviews['comments'] == '!') | (reviews['comments'] == '?') ]

,listing_id,id,date,reviewer_id,reviewer_name,comments
609,5178,458595941119654016,2021-09-24,409836786,Elya,.
8075,27644,503596934129974976,2021-11-25,38781491,Jay,.
9103,31994,180524495,2017-08-10,98704132,Martin (Aka Max),.
9106,31994,194910000,2017-09-17,99756545,Vardhan,.
9159,31994,366953207,2019-01-03,213464080,Glenda,.
...,...,...,...,...,...,...
1045288,593313785791808000,645676525698039040,2022-06-09,461019684,Devon,.
1046154,596511810816093056,618129884161660032,2022-05-02,455819359,Tatiana,.
1046404,597180195652908032,610077003524514944,2022-04-21,421646015,Guillermo,.
1047280,599993984917025024,623911135936929024,2022-05-10,46317236,Alexandre,.


## Lets start actually cleaning our data

In [14]:
reviews['comments'].dropna(inplace = True)

In [15]:
review_list = reviews['comments'].to_list()
review_list[0]

'Great host '

In [16]:
for index, string in enumerate(review_list):
  if type(string) != type('example'):
    review_list[index] = ""

In [17]:
review_string = ' '.join(review_list)
review_string[:100]

'Great host  Nice room for the price. Great neighborhood. John was very accommodating. Bottles of wat'

In [18]:
split_text = [review_string[i:i+800000] for i in range(0, len(review_string), 800000)]

In [ ]:
# Initialize spacy 'en_core_web_sm' model, keeping only tagger component needed for lemmatization

nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

docs = []
for piece in split_text:
    doc = nlp(piece)
    docs.append(doc)
    
merged = Doc.from_docs(docs)

In [ ]:
# Extract the lemma for each token and join
lemmatized_corpus = " ".join([token.lemma_.lower() for token in doc if not (token.is_punct or pattern.match(token.text))])

lemmatized_corpus_list = lemmatized_corpus.split(sep = "")

In [ ]:
len(lemmatized_corpus)

lemmatized_corpus_list

## Time to clean our text

In [ ]:
reviews_1000 = reviews.head(1000)

reviews_1000['cleaned_tokens'] = lemmatized_corpus_list